## Problem 1. PageRank cheat in the Buckley - Ostgus model

Link farm is the construction presented in the figure. Show how the number of nodes in the farm influence the node  $v$'s position in ranking by PageRank.

<img src="farm.png"> 


In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generateSimpleBuckleyOsthusGraph(a,n):
    G = nx.MultiDiGraph()
    G.add_node(0)
    G.add_edge(0,0)
    p = [a+1]
    while G.number_of_nodes() < n:
        new_node = G.number_of_nodes()
        G.add_node(new_node)
        p.append(a)
        probabilities = np.array(p)/((a+1.0)*new_node+a)
        destination = np.random.choice(G.nodes(),1,p=probabilities)[0]
        G.add_edge(new_node,destination)
        p[destination] += 1
    return G


def generateBuckleyOsthusGraph(a, n, m):
    G1mn = generateSimpleBuckleyOsthusGraph(a, m*n)
    G = nx.MultiDiGraph()
    for u, v in G1mn.edges():
        u_new, v_new = u // m, v // m
        G.add_edge(u_new, v_new)
    return G

In [ ]:
a = 0.27
n = 1000
m = 5
G = generateBuckleyOsthusGraph(a, n, m)
G = nx.DiGraph(G)

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
pr = nx.pagerank(G, alpha = 0.85)
pr_sorted = sorted(pr.items(), key = lambda x:x[1], reverse=True)
pr_sorted

In [ ]:
ranks_to_nodes = {rank: pr_sorted[rank][0] for rank in range(len(pr_sorted)) }
ranks_to_nodes

In [ ]:
nodes_to_ranks = {v:k for k,v in ranks_to_nodes.items()}

In [ ]:
nodes_to_ranks[800]

In [ ]:
G_new = G.copy()
k = 10
node_v = 800
for i in range(k):
    G_new.add_edge(f'v{i}',node_v)
    G_new.add_edge(node_v,f'v{i}')

In [ ]:
G_new.number_of_nodes()

In [ ]:
pr_new = nx.pagerank(G_new, alpha = 0.85)
pr_sorted_new = sorted(pr_new.items(), key = lambda x:x[1], reverse=True)
ranks_to_nodes_new = {rank: pr_sorted_new[rank][0] for rank in range(len(pr_sorted_new)) }
nodes_to_ranks_new = {v:k for k,v in ranks_to_nodes_new.items()}
nodes_to_ranks_new[800]

## Problem 2. PageRank in the Avrachenkov model

Implement generation of the random graph in the Avrachenkov's concretization of the Barabasi - Albert model.

At each time point we add one node and  $m$ edges.

We start from node 0 without edges, but with weight equal to $m$. 

Node 1 makes $m$ links to 0. Weight of the node 0 becomes equal to $2m$, node 1 weight is now $m$.

The next nodes make $m$ links independently, each with probability proportional to the nodes weights (= sum of their incoming links and the initial weight $m$):
$$
P(n+1\to i)=\frac{\mathrm{indeg}\,i + m}{\sum\limits_{k=0}^n(\mathrm{indeg}\,k + m)}=\frac{\mathrm{indeg}\,i + m}{2mn + m}.
$$

Using the Avrachenkov's theorem, for the fixed $n$ plot the estimate for the cumulative PageRank distribution function $P(\pi_v > x)$. 

Build the graph on  $n=1000$ nodes and with $m=5$. Plot the theoretical estimate and numerical results for the PageRank distribution with $\alpha=0.85$.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pylab

In [ ]:
import random

def generateGraph(n, m):
    G = nx.MultiDiGraph()
    G.add_node(0)
    G.add_node(1)
    for x in range(m):
        G.add_edge(1, 0)
    repeated_nodes = [0] * (2 * m) + [1] * m
    while G.number_of_nodes() <= n:
        new_node = G.number_of_nodes()
        G.add_node(new_node)
        dests = []
        for x in range(m):
            destination = random.choice(repeated_nodes)
            G.add_edge(new_node, destination)
            dests.append(destination)
        repeated_nodes.extend(dests)
        repeated_nodes.extend([new_node] * m)
    return G

In [ ]:
N = 1000
m = 5
G = generateGraph(N, m)
# we add (0, 0) not to count nodes with zero outdegrees
G.add_edge(0, 0)

we use pagerank_scipy, as this implementation works with Multi(Di)Graphs. For multigraphs the weight between two nodes is set to be the sum of all edge weights between those nodes.

In [ ]:
alpha = 0.85
P = nx.pagerank_scipy(G, alpha)

In [ ]:
pagerank_distr = []
xs = []
for x in sorted(P.values()):
    xs.append(x)
    pagerank_distr.append(len([v for v in G.nodes() if P[v] >= x]) / float(N))

According to the Avrachenkov's theorem when $ i > 0 $ the expected value of node $i$'s PageRank is calculated as follows:
$$
{\bf E}\pi_i(n) 
\approx \frac{1-\alpha}{1+n}\left(\frac{1}{1+\alpha} + \frac{\alpha}{1+\alpha}\left(i+\frac{1}{2}\right)^{-\frac{1+\alpha}{2}} \left(n+\frac{1}{2}\right)^{\frac{1+\alpha}{2}}\right).
$$

When $n$ is fixed: ${\bf E}\pi_i=f(i)$, where $f(i)$ is decreasing function. Therefore,
\begin{equation*}
\begin{split}
\textbf{P}(\pi_i<x)&=\textbf{P}(i>f^{-1}(x))\approx\textbf{P}\left(i>\left(\left(\frac{1+n}{1-\alpha}x-\frac{1}{1+\alpha}\right)\frac{1+\alpha}{\alpha}\left(n+\frac{1}{2}\right)^{-\frac{1+\alpha}{2}}\right)^{\frac{-2}{1+\alpha}}-\frac12\right)=\\
&=1-\frac{1}{n}\cdot\left(\left(\left(\frac{1+n}{1-\alpha}x-\frac{1}{1+\alpha}\right)\frac{1+\alpha}{\alpha}\left(n+\frac{1}{2}\right)^{-\frac{1+\alpha}{2}}\right)^{\frac{-2}{1+\alpha}}-\frac12\right)=\\
&=1+\frac{1}{2n}-\left(1+\frac{1}{2n}\right)\alpha^{\frac{2}{\alpha+1}}\left(\frac{1+\alpha}{1-\alpha}(n+1)x-1\right)^{\frac{-2}{1+\alpha}}.
\end{split}
\end{equation*}

Let us note that $\textbf{P}(\pi_i>=x)=1-\textbf{P}(\pi_i<x)$. 

In [ ]:
theoretical_est = []
xs2 = []
for x in np.arange(min(P.values()), max(P.values()), 0.001):
    val = (1 + 1 / (2 * N)) * alpha ** (2 / (alpha + 1))
    val *= ((N + 1) * (1 + alpha) * x / (1 - alpha) - 1) ** (-2 / (alpha + 1))
    val = 1 + 1 / (2 * N) - val
    xs2.append(x)
    theoretical_est.append(1 - val)

In [ ]:
import matplotlib.pyplot as plt

pylab.rcParams['figure.figsize'] = 10, 10
plt.loglog(xs, pagerank_distr, c='r', marker='+', ls='None', markersize=8)
plt.loglog(xs2, theoretical_est)
plt.show()

## Problem 3. Bollobas - Borgs - Riordan -Chayes model

In [ ]:
def GenerateBBRCGraph(delta_out, delta_in, alpha, beta, gamma, time):
    G = nx.MultiDiGraph()
    G.add_edge(0,0)
    
    for t in range(1,time):
        event = random.choices(["node_and_outedge","node_and_inedge","new_edge"],
                               weights = [alpha,gamma,beta])[0]
        
        if event == "node_and_outedge":
            new_node = G.number_of_nodes()
            in_degrees = dict(G.in_degree)
            weights = [in_deg + delta_in for in_deg in in_degrees.values()]
            dest = random.choices(range(G.number_of_nodes()), weights = weights)[0]
            G.add_edge(new_node, dest)
        
        if event == "node_and_inedge":
            new_node = G.number_of_nodes()
            out_degrees = dict(G.out_degree)
            weights = [out_deg + delta_out for out_deg in out_degrees.values()]
            dest = random.choices(range(G.number_of_nodes()), weights = weights)[0]
            G.add_edge(dest,new_node)
        
        if event == "new_edge":
            in_degrees = dict(G.in_degree)
            in_weights = [in_deg + delta_in for in_deg in in_degrees.values()]
            in_dest = random.choices(range(G.number_of_nodes()), weights = in_weights)[0]
            
            out_degrees = dict(G.out_degree)
            out_weights = [out_deg + delta_out for out_deg in out_degrees.values()]
            out_dest = random.choices(range(G.number_of_nodes()), weights = out_weights)[0]
            
            G.add_edge(out_dest, in_dest)
    return G

In [ ]:
delta_out = 0.3 
delta_in = 0.3
beta = 0.9
alpha, gamma = (1 - beta) / 2, (1 - beta) / 2
time = 1000

In [ ]:
G = GenerateBBRCGraph (delta_out, delta_in, alpha, beta, gamma, time)

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

In [ ]:
densities = []
betas = [x/10 for x in range(1,10)]
for beta in betas:
    alpha, gamma = (1 - beta) / 2, (1 - beta) / 2
    G = GenerateBBRCGraph (delta_out, delta_in, alpha, beta, gamma, time)
    dens = G.number_of_edges() / G.number_of_nodes()
    densities.append(dens)

In [ ]:
plt.plot(betas, densities)

In [ ]:
beta = 0.3
alpha, gamma = (1 - beta) / 2, (1 - beta) / 2
time = 5000
G = GenerateBBRCGraph (delta_out, delta_in, alpha, beta, gamma, time)

In [ ]:
from collections import Counter
in_degrees = dict(G.in_degree)
in_degrees_counts = Counter(list(in_degrees.values()))

In [ ]:
plt.loglog(list(in_degrees_counts.keys()), list(in_degrees_counts.values()), ls='None', marker='.')
plt.xlabel('In-degree')
plt.ylabel('Counts')
plt.title('In-degree distribution')

In [ ]:
out_degrees = dict(G.out_degree)
out_degrees_counts = Counter(list(out_degrees.values()))

In [ ]:
plt.loglog(list(out_degrees_counts.keys()), list(out_degrees_counts.values()), ls='None', marker='.')
plt.xlabel('Out-degree')
plt.ylabel('Counts')
plt.title('Out-degree distribution')

In [ ]:
from scipy.optimize import curve_fit
import numpy as np

def power_law(d, c, gamma):
    return c / (d ** gamma)

degrees_list = list(in_degrees_counts.keys())
counts_list = list(in_degrees_counts.values())

h1, h2 = 8, 100

xdata = np.array([x for x in degrees_list if x <= h2 and x>= h1])
ydata = np.array([counts_list[i] for i in range(len(counts_list)) 
                  if degrees_list[i] >= h1 and degrees_list[i] <= h2])
popt, pcov = curve_fit(power_law, xdata, ydata)
c, gamma = popt

In [ ]:
gamma

In [ ]:
plt.loglog(list(in_degrees_counts.keys()), list(in_degrees_counts.values()), ls='None', marker='.')
estimated = [power_law(x, c, gamma) for x in degrees_list]
plt.loglog(degrees_list, estimated, ls = '--', color = 'r')
plt.xlabel('In-degree')
plt.ylabel('Counts')
plt.title('In-degree distribution')

In [ ]:
degrees_list = list(out_degrees_counts.keys())
counts_list = list(out_degrees_counts.values())

h1, h2 = 8, 100

xdata = np.array([x for x in degrees_list if x <= h2 and x>= h1])
ydata = np.array([counts_list[i] for i in range(len(counts_list)) 
                  if degrees_list[i] >= h1 and degrees_list[i] <= h2])
popt, pcov = curve_fit(power_law, xdata, ydata)
c, gamma = popt

In [ ]:
gamma

In [ ]:
plt.loglog(list(out_degrees_counts.keys()), list(out_degrees_counts.values()), ls='None', marker='.')
estimated = [power_law(x, c, gamma) for x in degrees_list]
plt.loglog(degrees_list, estimated, ls = '--', color = 'r')
plt.xlabel('Out-degree')
plt.ylabel('Counts')
plt.title('Out-degree distribution')

In [ ]:
n = len(G.nodes)
n_edges = len(G.edges)
print(f'number of nodes = {n}')
print(f'number of edges = {n_edges}')

In [ ]:
wcc_list = list(nx.weakly_connected_components(G))
wcc_list = sorted(wcc_list, key = len, reverse=True)

print(f'number of weakly connected componets = {len(wcc_list)}')
print(f'size of GWCC = {len(wcc_list[0])/len(G.nodes)}')

In [ ]:
scc_list = list(nx.strongly_connected_components(G))
scc_list = sorted(scc_list, key = len, reverse=True)

print(f'number of strongly connected componets = {len(scc_list)}')
print(f'size of GSCC = {len(scc_list[0])/len(G.nodes)}')
print(f'size of second SCC = {len(scc_list[1])/len(G.nodes)}')

## Problem 4. 
The graph generated using the Bollobas - Borgs - Riordan -Chayes model with $\alpha = \beta = 0.5$, $\gamma = \delta_{out} = 0$ is presented in the file graph.txt. $\delta_{in}$ is unknown.

The order of edges and vertex numbers in the file corresponds to the order in which they actually appeared.
The first number in the line is the beginning of the edge, the second is the end of the edge. Initial graph - triangle

[(0, 1), (1, 2), (2, 0)].

Plot the dependence of the probability of the given graph for the given values of the parameters on $\delta_{in}$ and find the optimal value at which the appearance of such a graph is most likely.

#### Solution
Consider all new edges of the graph in the order of their appearance and find the probability of the appearance of each edge at the corresponding moment in time. According to the definition of the model, a new vertex is added with a probability $\alpha = 0.5$ and not added with probability $\beta=0.5.$  (in this case the edge is drawn between the existing nodes). 

As  $\gamma = \delta_{out}=0,$ while building  $G(t + 1)$
from the graph $G(t)$ on $n_t$ nodes the probability to add new node $u$ and outgoing edge from it to some existing node $v$ ребро есть
$$
\frac{1}{2}\cdot\frac{\mathrm{indeg}\,v + \delta_{in}}{t+\delta_{in}n_t}.
$$
The probability to add the new edge between existing nodes  $u$ and $v$ is equal to
$$
\frac{1}{2}\cdot\frac{\mathrm{outdeg}u}{t}\cdot\frac{\mathrm{indeg}\,v + \delta_{ in} }{t+\delta_{in}n_t}.
$$


The probability of having each particular graph $G(t)$ is equal to $\prod\limits_ip_i,$ where $p_i$ is determined by these formulas depending on whether the vertex $u_i$ is new. 

Given a particular graph, we need to maximize this probability (likelihood). In order not to work with very small numbers, we will instead of the problem $\prod\limits_ip_i \to \max\limits_{\delta_{in}}$ solve the equivalent one $\sum\limits_i\ln p_i \to \max\limits_{\delta_{in}}.$

Let us note that $\ln \frac{1}{2}$, $\ln \frac{\mathrm{outdeg}u}{t}$ do not depend on $\delta_{in},$ therefore, the problem is to maximize
$$
\sum\limits_i\ln\frac{\mathrm{indeg}\,v + \delta_{in}}{t+\delta_{in}n_t}=\sum\limits_i\left(\ln(\mathrm{indeg}\,v + \delta_{in}) - \ln(t+\delta_{in}n_t)\right).
$$

In [ ]:
import networkx as nx
G = nx.MultiDiGraph()
G.add_edges_from([(0, 1), (1, 2), (2, 0)])

log_params = []
t = 3
for line in open("graph.txt"):
    u, v = line.strip().split('\t')
    u, v = int(u), int(v)
    log_params.append((G.in_degree(v), t, G.order()))
    t += 1
    G.add_edge(u, v)


In [ ]:
G.edges()

In [ ]:
from numpy import log, arange

def f(log_params, delta_in):
    res = 0
    for indeg, t, nt in log_params:
        res += log(indeg + delta_in) - log(t + delta_in * nt)
    return res

delta_in_values = []
f_values = []
for delta_in in arange(0.001, 1, 0.001):
    f_values.append(f(log_params, delta_in))
    delta_in_values.append(delta_in)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(delta_in_values[20:],f_values[20:])
plt.show()

In [ ]:
max(f_values)

In [ ]:
max(zip(delta_in_values,f_values), key=lambda x: x[1])